In [31]:
# Importing modules...
import numpy as np
import socket		

In [32]:
print('> Creating server socket...')

# Try to create server socket
try: 
    server_socket = socket.socket(socket.AF_INET,    # Internet
                                  socket.SOCK_DGRAM) # UDP
    print('> Socket successfully created')
    
except socket.error as err: 
    print(f'> Socket creation failed with error {err}\n')
    print('> Quitting server...')
    exit()

UDP_IP = '127.0.0.1' # Server IP
UDP_PORT = 5005      # UDP Port
buffer_size = 1024   # In bytes

server_socket.bind((UDP_IP, UDP_PORT))
print(f'> Server bound to port {UDP_PORT}')

> Creating server socket...
> Socket successfully created
> Server bound to port 5005


In [33]:
n_clients = 4
address_list = {}

print('> Waiting for clients...')

# Address lookup 
while len(address_list.keys()) < n_clients: 
    message_bytes, address = server_socket.recvfrom(buffer_size)
    ID = int(message_bytes.decode())

    address_list[address] = ID

    print(f'\tClient {ID} Connected')

print('> All clients connected!')

> Waiting for clients...
	Client 0 Connected
	Client 1 Connected
	Client 2 Connected
	Client 3 Connected
> All clients connected!


In [34]:
timeout = 10 # In seconds
server_socket.settimeout(timeout) # Set server timeout
print(f'> Server timeout set to {timeout} seconds\n')

PTS = 0.0 # Presentation Time Stamp
target_PTS  = 1.0 # Only receive messages until this timestamp 

while PTS < target_PTS:  
    # Wait for message - Event guided!
    try:
        message_bytes, address = server_socket.recvfrom(buffer_size)

    except socket.timeout as err:
        print('\n> Server timed out')
        break # Close loop due to timeout
    
    # Decode message 
    message = np.frombuffer(message_bytes, dtype=np.float64)

    detected_blobs = message[:-1].reshape(-1, 2) # All but last element are the blob coordinates
    PTS = message[-1] # Last element contains PTS

    print(f'> Received message from Client {address_list[address]} ({address[0]}, {address[1]}):')
    print(f'\tDetected Blobs - {PTS :.3f} s')
    print('\t' + str(detected_blobs).replace('\n', '\n\t'))


print('\nConnection Endend.')

> Server timeout set to 10 seconds

> Received message from Client 0 (127.0.0.1, 52315):
	Detected Blobs - 0.000 s
	[[359.30000019 246.30000019]]
> Received message from Client 0 (127.0.0.1, 52315):
	Detected Blobs - 0.050 s
	[[359.30000019 243.30000019]]
> Received message from Client 1 (127.0.0.1, 52316):
	Detected Blobs - 0.050 s
	[[359.30000019 243.30000019]]
> Received message from Client 2 (127.0.0.1, 65185):
	Detected Blobs - 0.050 s
	[[359.30000019 243.30000019]]
> Received message from Client 3 (127.0.0.1, 65186):
	Detected Blobs - 0.050 s
	[[359.30000019 243.30000019]]
> Received message from Client 0 (127.0.0.1, 52315):
	Detected Blobs - 0.100 s
	[[359.37589407 240.7321701 ]]
> Received message from Client 1 (127.0.0.1, 52316):
	Detected Blobs - 0.100 s
	[[359.08844566 240.7321701 ]]
> Received message from Client 2 (127.0.0.1, 65185):
	Detected Blobs - 0.100 s
	[[359.30000019 240.30000019]]
> Received message from Client 3 (127.0.0.1, 65186):
	Detected Blobs - 0.100 s
	[[35